In [ ]:
'''
문제2) Logistic regression
대학원 입학자료를 이용하여 특정 조건에 따른 입학여부를 예측하는 프로그램을 
TensorFlow를 이용하여 작성한다.
제공된 CSV 파일로부터 데이터를 loading한 후 학습과정을 진행하고
특정 조건(GRE,GPA,RANK)에 따른 합격여부를 예측한다.
'''

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
import statsmodels.api as sm

warnings.filterwarnings(action="ignore")

    # 1. Data Loading
data = pd.read_csv("./Data/Admission/admission.csv",sep=",")
data = data.dropna(how="any", inplace= False)
data.head()
data_x = data[["gre","gpa","rank"]]
data_y = data["admit"]
data_y = pd.DataFrame(data_y,columns = ["admit"])

rank_dummies = pd.get_dummies(data_x["rank"], prefix ="rank")

data_x = data_x.join(rank_dummies)
data_x.drop("rank", axis = 1, inplace = True)

#nomalization

data_x = MinMaxScaler().fit_transform(data_x[["gre", "gpa","rank_1","rank_2","rank_3","rank_4"]].values)
data_y = MinMaxScaler().fit_transform(data_y["admit"].values.reshape(-1,1))
   
    # 2. Training data set
# 학습용, 테스트용 데이터 분할
rows,_ = data_x.shape
Tr_x = data_x[:int(rows*0.7)]
Te_x = data_x[int(rows*0.7):]


rows,_ = data_y.shape
Tr_y = data_y[:int(rows*0.7)]
Te_y = data_y[int(rows*0.7):]


#3. PlaceHolder
# Tr_x .shape #(280, 6)
X = tf.placeholder(shape = [None,6], dtype = tf.float32)
Y = tf.placeholder(shape = [None,1], dtype = tf.float32)

#4. Weight & bias
W = tf.Variable(tf.random_normal([6,1]), name = "weight")
b = tf.Variable(tf.random_normal([1]), name = "bias")

#5. Hypothesis
logit = tf.matmul(X,W)+b
H = tf.sigmoid(logit)

#6. Cost function
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logit, labels = Y))


#7. Train
train = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(cost)


#8. session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#9. 학습
for step in range(10000):
    _,cost_val = sess.run([train,cost], feed_dict={X:Tr_x, Y:Tr_y})
    if step % 1000 == 0:
        print("Cost:{}".format(cost_val))
        
        #10. accuracy 측정
predict = tf.cast(H>0.5, dtype = tf.float32)
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.float32))

print("정확도 :{}".format(sess.run(accuracy, feed_dict = {X:Te_x, Y:Te_y})))

#prediction 
display(data[:1])
input_gre = input("GRE :")
input_gpa = input("GPA :")
input_rank = input("Rank :")
dict_p = {"GRE":[input_gre, None,None,None,None]
          , "GPA":[input_gpa,None,None,None,None]
          , "Rank": [input_rank, "1","2","3","4"]}
data_p = pd.DataFrame(dict_p)
display(data_p)
rank_dummies1 = pd.get_dummies(data_p["Rank"], prefix ="Rank")
data_p = data_p.join(rank_dummies1)
data_p.drop("Rank", axis = 1, inplace = True)
print(data_p[:1])
data_p = MinMaxScaler().fit_transform(data_p[["GRE", "GPA","Rank_1","Rank_2","Rank_3","Rank_4"]].values)
data_p = data_p[:1]

result = sess.run(H,feed_dict ={X:data_p})
if result>0.5:
    print("합격할 것으로 예상됩니다." )
else:
    print("합격하지 못 할 것 같습니다." )

Cost:0.6035994291305542
Cost:0.5733141303062439
Cost:0.5615683197975159
Cost:0.5562935471534729
Cost:0.5536196827888489
Cost:0.5521223545074463
Cost:0.5512091517448425
Cost:0.5506073236465454
Cost:0.550182044506073
Cost:0.5498629808425903
정확도 :0.7166666388511658


,admit,gre,gpa,rank
0,0,380,3.61,3


GRE :380
GPA :3.61
Rank :3


,GRE,GPA,Rank
0,380,3.61,3
1,None,None,1
2,None,None,2
3,None,None,3
4,None,None,4


   GRE   GPA  Rank_1  Rank_2  Rank_3  Rank_4
0  380  3.61       0       0       1       0
합격하지 못 할 것 같습니다.
